# Performance of Shapley Values

### 1. **First we need a complex model**

In [ ]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# Load the breast cancer dataset
data = load_breast_cancer()

# Access the features and target
X = data.data
y = data.target

# Access the feature names and target names
feature_names = data.feature_names
target_names = data.target_names

# Create a DataFrame for the features
df = pd.DataFrame(X, columns=feature_names)

# Add the target column to the DataFrame
df['target'] = y

# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

# We keep only the four first columns:

X_train = X_train.iloc[:,:4]
X_test = X_test.iloc[:, :4]

# Create a DataFrame LightGBM for the train & test dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Define the model parameters
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose': -1
}

# Train the model
num_round = 100
model = lgb.train(params, train_data, num_round, valid_sets=[test_data], )
# Predictions on the test set
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Convert the probabilities in binary classes
y_pred_binary = [1 if pred > 0.5 else 0 for pred in y_pred]

# Compute the accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy}')

Accuracy: 0.8771929824561403


### 2. **SHAP values**

In [22]:
import shap

# Create the Kernel SHAP explainer (LIME + kernel)
explainer = shap.KernelExplainer(model.predict, X_train, link ='logit', algorithm ='linear')

# Compute the SHAP values for each instance of X_test. 
# The SHAP values is estimated based on a sample of size 100, that will be averaged.
shap_values = explainer.shap_values(X_test, nsamples = 100)

Using 455 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/114 [00:00<?, ?it/s]

The use of Shapley values to explain the predictions of a complex model is a powerful method for making models interpretable.  

However, evaluating the performance of Shapley values in this context can be a bit more subtle than evaluating the performance of a traditional machine learning model.  

Here are some approaches to evaluate the performance of Shapley values:

### 3. **Performances of Shapley ?**

#### 3.1. Consistency with Expectations

Check if the Shapley values align with your expectations based on your domain knowledge.  
For example, if you know that a certain feature should have a significant impact on the prediction, the Shapley values should reflect that.

#### 3.2. Comparison with Other Explanation Methods

Compare the Shapley values with other model explanation methods, such as LIME (Local Interpretable Model-agnostic Explanations) or feature weights in simpler models.  
If the explanations are consistent across different methods, it strengthens confidence in the Shapley values.

#### 3.3. Quantitative Evaluation

Use quantitative metrics to evaluate the performance of Shapley values. Here are some commonly used metrics:

##### 3.3.1. Fidelity  

Measure the fidelity of the explanations by comparing the predictions of the original model with the predictions of a simplified model based on the Shapley values.  
High fidelity means that the explanations are close to the actual predictions of the model.

```python
import numpy as np
import shap

# Supposons que vous avez un modèle 'model' et des données 'X'
explainer = shap.Explainer(model, X)
shap_values = explainer(X)

# Calculer la fidélité
def fidelity(model, X, shap_values):
    original_predictions = model.predict(X)
    shap_predictions = np.sum(shap_values.values, axis=1) + shap_values.base_values
    return np.mean((original_predictions - shap_predictions) ** 2)

fidelity_score = fidelity(model, X, shap_values)
print(f"Fidelity score: {fidelity_score}")
```

#### b. **Stabilité**
Mesurez la stabilité des Shapley values en calculant les valeurs pour des échantillons légèrement perturbés. Les Shapley values devraient être relativement stables pour des échantillons similaires.

```python
def stability(model, X, shap_values, perturbation_size=0.01):
    perturbed_X = X + np.random.normal(0, perturbation_size, X.shape)
    perturbed_shap_values = explainer(perturbed_X)
    return np.mean((shap_values.values - perturbed_shap_values.values) ** 2)

stability_score = stability(model, X, shap_values)
print(f"Stability score: {stability_score}")
```

### 4. **Évaluation qualitative**
Impliquez des experts du domaine pour évaluer qualitativement les explications fournies par les Shapley values. Les experts peuvent fournir des insights sur la pertinence et la précision des explications.

### 5. **Visualisation**
Utilisez des visualisations pour inspecter les Shapley values. Les graphiques de dépendance, les graphiques de résumé et les graphiques de force peuvent aider à comprendre l'impact des différentes caractéristiques sur les prédictions.

```python
# Graphique de résumé
shap.summary_plot(shap_values, X)

# Graphique de dépendance
shap.dependence_plot("feature_name", shap_values.values, X)

# Graphique de force
shap.force_plot(explainer.expected_value, shap_values.values, X)
```

### Conclusion
Évaluer la performance des Shapley values dans le contexte de l'interprétabilité des modèles nécessite une combinaison de méthodes quantitatives et qualitatives. En utilisant des métriques comme la fidélité et la stabilité, en comparant avec d'autres méthodes d'explication, et en impliquant des experts du domaine, vous pouvez obtenir une évaluation complète de la performance des Shapley values.

# Detecting bias with SHAP

# Detecting bias with SHAP

```python
from scipy import stats

gender_cols = [col for col in X_train.columns if 'Gender' in col] + ["Trans"]

gender_features_locs = [X_train.columns.get_loc(col) for col in gender_cols]

def describe_stats(gender_col):
    s = stats.describe(shap_values[X_train[gender_col]][:,gender_features_locs].sum(axis=1))
    return (gender_col, int(s.nobs), float(s.minmax[0]), float(s.minmax[1]), float(s.mean))

display(pd.DataFrame([describe_stats(col) for col in gender_cols], ["gender_col", "n", "min", "max", "mean"]))
```

## Detecting bias